# TFLite → .task Bundle

Bundles `.tflite` model with tokenizer into `.task` format for MediaPipe.

**Pipeline:**
1. [functiongemma_finetuning.ipynb](https://github.com/DenisovAV/flutter_gemma/blob/main/colabs/functiongemma_finetuning.ipynb) - Fine-tune model ✅
2. [functiongemma_to_tflite.ipynb](https://github.com/DenisovAV/flutter_gemma/blob/main/colabs/functiongemma_to_tflite.ipynb) - Convert to TFLite ✅
3. **This notebook** - Bundle as .task for Flutter

**Requirements:**
- `.tflite` file from Step 2
- `tokenizer.model` file
- Both saved to Google Drive

**Note:** This notebook uses ONLY mediapipe (no ai-edge-torch) to avoid conflicts.

## Step 1: Install MediaPipe

MediaPipe is Google's library for on-device ML inference.
We use the `bundler` module to package TFLite model with tokenizer into a single `.task` file.

**Important:** 
- We use mediapipe version 0.10.20
- numpy<2.1 is required for compatibility
- **RESTART RUNTIME** after this step!

In [ ]:
# =============================================================================
# Step 1: Install mediapipe (clean environment, no ai-edge-torch)
# =============================================================================
!pip install mediapipe==0.10.20 -q

# CRITICAL: Install numpy<2.1 AFTER mediapipe (fixes binary incompatibility)
!pip install "numpy<2.1" --force-reinstall -q

print("Installed:")
!pip show mediapipe | grep Version
!pip show numpy | grep Version

print("\n⚠️  RESTART RUNTIME after this step! (Runtime → Restart session)")
print("Then run cells starting from Step 2.")

## Step 2: Load Files from Google Drive

Load files created in the previous notebook `functiongemma_to_tflite.ipynb`:
- **TFLite model** — converted FunctionGemma 270M
- **Tokenizer** — SentencePiece tokenizer from the base model

If tokenizer is not found on Drive, we download it from HuggingFace.

In [ ]:
# =============================================================================
# Step 2: Load TFLite and tokenizer from Google Drive
# =============================================================================
from google.colab import drive
import os

# Verify bundler works after restart
from mediapipe.tasks.python.genai import bundler
print("✅ bundler module: OK\n")

drive.mount('/content/drive')

DRIVE_INPUT_DIR = "/content/drive/MyDrive/flutter_gemma_models"

# TFLite model - specific file
TFLITE_MODEL = f"{DRIVE_INPUT_DIR}/functiongemma-flutter_q8_ekv1024.tflite"

if not os.path.exists(TFLITE_MODEL):
    raise FileNotFoundError(
        f"File not found: {TFLITE_MODEL}\n"
        "Run functiongemma_to_tflite.ipynb first!"
    )

print(f"TFLite: {TFLITE_MODEL}")
print(f"Size: {os.path.getsize(TFLITE_MODEL) / 1e6:.1f} MB")

# Tokenizer
TOKENIZER_PATH = f"{DRIVE_INPUT_DIR}/tokenizer.model"
if not os.path.exists(TOKENIZER_PATH):
    print("\nTokenizer not found locally, downloading from HuggingFace...")
    from huggingface_hub import hf_hub_download
    TOKENIZER_PATH = hf_hub_download(
        repo_id="google/functiongemma-270m-it",
        filename="tokenizer.model"
    )

print(f"Tokenizer: {TOKENIZER_PATH}")

## Step 3: Create .task Bundle

`.task` is MediaPipe's format that combines model and tokenizer into a single file.

**Key parameters:**
- `stop_tokens` — tokens that make the model stop generation
- `<start_function_response>` — **critical** stop token for FunctionGemma
  (model must stop after function call and wait for the result)
- `prompt_prefix/suffix` — wrapping for user input

Documentation: [FunctionGemma Formatting](https://ai.google.dev/gemma/docs/functiongemma/formatting-and-best-practices)

In [ ]:
# =============================================================================
# Step 3: Create MediaPipe .task bundle
# =============================================================================
from mediapipe.tasks.python.genai import bundler

TASK_OUTPUT = "functiongemma-flutter_q8_ekv1024.task"

# FunctionGemma-specific bundle configuration
# Per Google docs: <start_function_response> IS a stop token for inference
# https://ai.google.dev/gemma/docs/functiongemma/formatting-and-best-practices
config = bundler.BundleConfig(
    tflite_model=TFLITE_MODEL,
    tokenizer_model=TOKENIZER_PATH,
    start_token="<bos>",
    stop_tokens=[
        "<end_of_turn>",
        "<start_function_response>",  # CRITICAL: stop after function call
    ],
    output_filename=TASK_OUTPUT,
    # EMPTY prefix/suffix - FunctionGemma format is handled in Dart code
    # (needs <start_of_turn>developer FIRST, not user)
    prompt_prefix="",
    prompt_suffix="",
)

print("Bundle configuration:")
print(f"  Stop tokens: {config.stop_tokens}")
print(f"  Start token: {config.start_token}")
print(f"  Prompt prefix: {repr(config.prompt_prefix)}")
print(f"  Prompt suffix: {repr(config.prompt_suffix)}")

print("\nCreating .task bundle...")
bundler.create_bundle(config)

print(f"\n✅ .task file created!")
!ls -lah {TASK_OUTPUT}

## Step 4: Save and Download

Save the ready `.task` file:
1. To Google Drive — for future use
2. Download locally — to add to Flutter project

After downloading, place the file in `example/assets/models/`.

In [ ]:
# =============================================================================
# Step 4: Save to Google Drive and download
# =============================================================================
import shutil
from google.colab import files

task_size = os.path.getsize(TASK_OUTPUT) / 1e6
print(f"Task file: {TASK_OUTPUT} ({task_size:.1f} MB)")

# Save to Drive
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/flutter_gemma_models"
drive_path = f"{DRIVE_OUTPUT_DIR}/{TASK_OUTPUT}"
shutil.copy(TASK_OUTPUT, drive_path)
print(f"Saved: {drive_path}")

# Download
print(f"\nDownloading...")
files.download(TASK_OUTPUT)

print("\n" + "=" * 50)
print("DONE!")
print("=" * 50)

## Optional: Upload to HuggingFace Hub

Upload the `.task` file to HuggingFace for easy sharing and distribution.

**Setup:**
1. Create a new model repository on [huggingface.co](https://huggingface.co/new)
2. Add `HF_TOKEN` to Colab Secrets (🔑 icon in left panel)
3. Change `HUB_REPO_ID` to your repository
4. Uncomment and run the code below

In [ ]:
# =============================================================================
# Optional: Upload to HuggingFace Hub
# =============================================================================
# Uncomment the code below to upload

# from huggingface_hub import login, HfApi
# from google.colab import userdata
#
# # Login (uses token from Colab Secrets)
# HF_TOKEN = userdata.get('HF_TOKEN')
# login(token=HF_TOKEN)
#
# # Upload to HuggingFace
# HUB_REPO_ID = "your-username/functiongemma-flutter-task"  # Change this!
#
# api = HfApi()
# api.create_repo(repo_id=HUB_REPO_ID, exist_ok=True)
#
# # Create README with license information (required by Gemma Terms)
# README_CONTENT = f"""# FunctionGemma (.task format)
#
# Converted version of [google/functiongemma-270m-it](https://huggingface.co/google/functiongemma-270m-it) for MediaPipe LLM Inference.
#
# ## Usage
#
# This model is designed for the [flutter_gemma](https://github.com/DenisovAV/flutter_gemma) plugin.
#
# ## Modifications
#
# - Converted from SafeTensors to TFLite format using ai-edge-torch
# - Quantized to int8 (dynamic quantization)
# - Bundled with tokenizer as .task file using MediaPipe bundler
#
# ## Files
#
# - `{TASK_OUTPUT}` - MediaPipe .task bundle ({task_size:.0f} MB)
#
# ## License
#
# Gemma is provided under and subject to the Gemma Terms of Use found at https://ai.google.dev/gemma/terms
#
# ## Original Model
#
# - Source: [google/functiongemma-270m-it](https://huggingface.co/google/functiongemma-270m-it)
# - License: [Gemma License](https://ai.google.dev/gemma/terms)
# - Prohibited Use Policy: [Gemma Prohibited Use Policy](https://ai.google.dev/gemma/prohibited_use_policy)
# """
#
# # Upload README
# api.upload_file(
#     path_or_fileobj=README_CONTENT.encode(),
#     path_in_repo="README.md",
#     repo_id=HUB_REPO_ID,
# )
# print("✅ Uploaded: README.md")
#
# # Upload .task file
# api.upload_file(
#     path_or_fileobj=TASK_OUTPUT,
#     path_in_repo=TASK_OUTPUT,
#     repo_id=HUB_REPO_ID,
# )
# print(f"✅ Uploaded: {TASK_OUTPUT}")
#
# print(f"\n🎉 Model uploaded to: https://huggingface.co/{HUB_REPO_ID}")